In [2]:
import numpy as np
import pandas as pd

import sys, os
sys.path.append("../..")
sys.path.append("..")
sys.path.append(os.getcwd())
sys.path.append("../../..")
from sklearn.linear_model import Ridge
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso 
from sklearn.decomposition import PCA #USE PCA FOR PCR REGRESSION
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
import datetime
import math
from tslib.src import tsUtils
from tslib.src.synthcontrol.syntheticControl import RobustSyntheticControl
from tslib.tests import testdata

import statsmodels.api as sm
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.neural_network import MLPRegressor

import warnings
from scipy.linalg import LinAlgWarning
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import GridSearchCV

import pyro
import pyro.distributions as dist
import torch
import pyro.infer.mcmc as mcmc

First, we define a get_training_data function that extracts data from our cleaned data source stadium_county_df 
that we obtained in our reproduction file Synth_Control_Paper. 

In [69]:
#Stadium data for obtaining training data 
stadium_county_df = pd.read_csv('Cleaned_stadium_data.csv')


#Grouped data for running Bayesian SC
grouped_df = pd.read_csv('Grouped_df.csv')

county_covid = pd.read_csv('County_Covid_Data.csv')

county_covid['date'] = pd.to_datetime(county_covid['date'])
county_covid = county_covid.loc[~(county_covid['county'] == 'Unknown')]
county_covid = county_covid.loc[~(county_covid['cases'].isnull())]
county_covid['county'] = county_covid['county'].apply(lambda x: x.lower() if isinstance(x, str) else x)

# Ignore LinAlgWarning
warnings.filterwarnings("ignore", category=LinAlgWarning)
warnings.filterwarnings("ignore", category=UserWarning)


In [27]:
home_state_dict = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}


In [116]:
#Below function gets training X and training Y data for applying Bayesian Model 
def get_training_data(team_name_str, stadium_county_str, state_str, intervention_date, show_plot, week):

    stadium_county_str = [x for x in stadium_county_str if x != ""]
    state_str = [x for x in state_str if x != ""]
    intervention_date = [x for x in intervention_date if x != ""]
    #Convert to lower case to avoid case insensitivity later
    stadium_county_str = [s.lower() for s in stadium_county_str]
    #To find intervention_date, we want first entry that has numbers, since some in data is text only.

    for s in intervention_date:
        if any(c.isdigit() for c in s):
            intervention_date = s


    intervention_date = pd.to_datetime(intervention_date)


    #Convert State Acronym to full state name
    for state in range(len(state_str)):
        if state_str[state] in home_state_dict:
            state_str[state] = home_state_dict[state_str[state]]
            
            
    #print(stadium_county_df)
    #Find Synthetic Counties
    synthetic_counties = list(stadium_county_df.loc[stadium_county_df['Team'] == team_name_str]['Donor_Counties'])[0]#.copy()
    #synthetic_counties = list(synthetic_counties)
    synthetic_counties = eval(synthetic_counties)
    synthetic_counties = [s.lower() for s in synthetic_counties]
    n_donors = len(synthetic_counties)
    
    #print(synthetic_counties)

    #Find Dataframe of X and Y data
    #Special case where 
    
    
    if team_name_str == 'Washington':
        stadium_county_data = county_covid.loc[(county_covid['county'].isin(stadium_county_str)) | ((county_covid['county'].isin(synthetic_counties)) & (county_covid['state'] == 'Maryland'))]
    else:
        stadium_county_data = county_covid.loc[(county_covid['county'].isin(stadium_county_str) | (county_covid['county'].isin(synthetic_counties))) & (county_covid['state'].isin(state_str))]
    
    stadium_county_data = stadium_county_data.fillna(method='bfill')
    stadium_county_data['date'] = pd.to_datetime(stadium_county_data['date'], infer_datetime_format=True)
    
    earliest_date = list(stadium_county_data.loc[stadium_county_data['county'].isin(stadium_county_str)]['date'])[0]
    
    #Start training from the earliest date of when our stadium county data becomes available.
    stadium_county_data = stadium_county_data.loc[stadium_county_data['date'] >= earliest_date]
    
    
    #Total Pivot is pivot table cases for entire dataset, training pivot is the same but for < intervention date
    total_pivot = stadium_county_data.pivot_table(columns='county', values='cases', index= 'date').reset_index()
    total_pivot = total_pivot.loc[total_pivot['date'] >= earliest_date]
    
    #Sum up stadium counties for our prediction. 
    total_pivot['Stadium_County'] = total_pivot.apply(lambda row: row[stadium_county_str].sum(), axis=1)
    #total_pivot['Stadium_County'] = total_pivot.loc[:, total_pivot.columns == (stadium_county_str[0])]
    
    
    total_pivot.drop(stadium_county_str, axis=1, inplace=True)
    
    total_pivot.fillna(0, inplace=True)
    
    training_pivot = total_pivot.loc[total_pivot['date'] < intervention_date]
    
    training_dates = training_pivot['date']
    
    total_dates = total_pivot['date']
    
    test_pivot = total_pivot.loc[total_pivot['date'] >= intervention_date]
    
    test_pivot = test_pivot.drop(['date'], axis=1)
    
    training_pivot = training_pivot.drop(['date'], axis=1)
    
    total_pivot = total_pivot.drop(['date'], axis=1)
    
    
    X_train = training_pivot.loc[:, ~training_pivot.columns.isin(['Stadium_County'])]
    
    Y_train = training_pivot['Stadium_County']
    
    total_X = total_pivot.loc[:, ~total_pivot.columns.isin(['Stadium_County'])]
    total_Y = total_pivot['Stadium_County']
    
    test_X = test_pivot.loc[:, ~test_pivot.columns.isin(['Stadium_County'])]
    test_Y = test_pivot['Stadium_County']
    
    

    return X_train, Y_train, test_X, test_Y, total_dates

x_train, y_train, x_test, y_test, dates = get_training_data('Cincinnati',["Hamilton"] ,["OH"], ['10/04/2020'], True, 0)




Here, we create the Model structures, all using Pyro to sample. 

We incorporate a mask input to use for our population predictive checks. 

The 3 models are Bayesian PCA, Poisson Factorization, and Poisson Gamma Dynamical Systems

In [3]:
def bayesian_pca(data, latent_dim, mask): #Pass in mask directly into the model 
    # Define model parameters
    n, p = data.shape
    sigma = pyro.sample("sigma", dist.Uniform(0., 10.))
    mu = torch.zeros(p)
    covariance = sigma * torch.eye(p)
    
    Z_mean = pyro.param("Z_mean", torch.zeros(n, latent_dim))
    Z_cov = pyro.param("Z_cov", torch.eye(latent_dim))


    W_mean = pyro.param("W_mean", torch.zeros(latent_dim, p))
    W_cov = pyro.param("W_cov", torch.eye(p))
    
    
    Z_mean.data = torch.zeros(n, latent_dim)
    Z_cov.data = torch.eye(latent_dim)
    W_mean.data = torch.zeros(latent_dim, p)
    W_cov.data = torch.eye(p)

    Z_nl = pyro.sample("Z", dist.MultivariateNormal(Z_mean, Z_cov))
    W_lp = pyro.sample("W", dist.MultivariateNormal(W_mean, W_cov))
    
    mean = Z_nl @ W_lp

    #Implement mask on our data to avoid masked probabilities. 

    X = pyro.sample("X", dist.Normal(mean, sigma).mask(mask), obs=data)
    
    return X

def poisson_factorization(data, latent_dim, mask):
    # Define model parameters
    n, p = data.shape
    
    #Construct samples of F, G, and X. X is assumed to be poisson distribution of F * G
    
#     alpha = pyro.sample("alpha", dist.Gamma(1., 1.))
#     beta = pyro.sample("beta", dist.Gamma(1., 1.))
    F = pyro.sample("F", dist.Gamma(1, 1).expand([n, latent_dim]))
    G = pyro.sample("G", dist.Gamma(1, 1).expand([latent_dim, p]))
    

    #use our mask to mask data

    #Implement mask on our data to avoid those probabilities. 
#     with pyro.poutine.mask(mask=mask):
#         X = pyro.sample("X", dist.Poisson(F @ G), obs=data)x
    X = pyro.sample("X", dist.Poisson(F @ G).mask(mask), obs=data)
    
    return X

def poisson_gamma_dynamical_system(data, latent_dim, mask=None):
    # Define model parameters
    n, p = data.shape
    time_steps = n
    G_matrix = []

    # Priors for latent states
    alpha = pyro.sample("alpha", dist.Gamma(1., 1.))
    beta = pyro.sample("beta", dist.Gamma(1., 1.))
    
    g_previous = pyro.sample("g_0", dist.Gamma(alpha, beta).expand([latent_dim]))

    #F should have shape latent * p 
    F_matrix = pyro.sample("F", dist.Gamma(1., 1.).expand([latent_dim, p]))
    
    G_matrix.append(g_previous)
    
    
    #K by K transition matrix
    concentration = torch.tensor([1] * latent_dim, dtype=torch.float)
    

    transition_matrix = pyro.sample("transition_matrix", dist.Dirichlet(concentration).expand([latent_dim]))#K by K matrix, Dirichlet Prior

    # Transition dynamics for the latent states
    
    # Transition dynamics for the latent states
    for t in pyro.markov(range(1, time_steps)):
        alpha_t = transition_matrix @ g_previous.T # previous
        
        g_t = pyro.sample(f"g_{t}", dist.Gamma(alpha_t, beta))
        G_matrix.append(g_t)

        # Update the previous state
        g_previous = g_t

    G_matrix = torch.stack(G_matrix, dim=0)
    
    if mask is None:
        mask = torch.ones(data.shape).bool()
    #Apply masking if necessary 
    X = pyro.sample("X", dist.Poisson(G_matrix @ F_matrix).mask(mask), obs=data)
    
    return X

Now, we define the MCMC functions we use to sample from the posterior distribution for posterior inference for each
of our three models. We use the NUTS kernel and pyro's built in Monte Carlo Markov Chain process

In [4]:
def bayesian_pca_mcmc(data, latent_dim, warmup_steps, num_samples):
    #data = torch.tensor(data.values)
    pyro.clear_param_store()   
    kernel = mcmc.NUTS(bayesian_pca)
    mcmc_run = mcmc.MCMC(kernel, num_samples=num_samples, warmup_steps=warmup_steps)

    N, P = data.shape
    mask_rows = torch.randperm(N)[:20]
    mask_cols = torch.randperm(P)[:5]

    mask = torch.ones(N,P)

    for i in mask_rows:
        for j in mask_cols:
            mask[i,j] = 0
    random_mask = mask.bool() #Boolean Tensor for mask

    end_block_mask = torch.ones(N,P)
    end_block_mask[-30:, -3:] = 0
    end_block_mask = end_block_mask.bool()


    test_mask = torch.ones(N,P)
    test_mask = test_mask.bool()

    #Apply MCMC to our data
    mcmc_run.run(data, latent_dim, random_mask)

    pyro.clear_param_store()
    #print(dummy_train)
    
    posterior_samples = mcmc_run.get_samples()

    # Extract W, sigma, and Z samples
    W_samples = posterior_samples["W"]
    sigma_samples = posterior_samples["sigma"]
    Z_samples = posterior_samples["Z"]
    
    return W_samples, sigma_samples, Z_samples

def poisson_mcmc(training_data, latent_dim, warmup_steps, samples, mask_type="Random"): #Default masking type is Random
    #Runs MCMC on the poisson model, returns F and G Samples

    #training_data = torch.tensor(training_data.values)
    #Is our latent dimension here also deterined as a hyperparameter? 
    model = poisson_factorization


    #Create mask for training/testing purposes
    N, P = training_data.shape
    
    if mask_type == "Random":
        mask_rows = torch.randperm(N)[:20]
        mask_cols = torch.randperm(P)[:5]

        mask = torch.ones(N,P)

        for i in mask_rows:
            for j in mask_cols:
                mask[i,j] = 0

        mask = mask.bool() #Random column and rows mask
        
    elif mask_type == "End":

        #TEST END BLOCK MASK
        end_block_mask = torch.ones(N,P)
        end_block_mask[-30:, -3:] = 0
        mask = end_block_mask.bool()
        
    elif mask_type == "None":
        mask = torch.ones(N,P).bool()
    else:
        print("Invalid Mask")
        return

    # Run MCMC
    kernel = mcmc.NUTS(model)
    mcmc_run = mcmc.MCMC(kernel, num_samples=samples, warmup_steps=warmup_steps)

    #Run MCMC process on our data with given Latent dimension
    mcmc_run.run(training_data, latent_dim, mask)
    
    posterior_samples = mcmc_run.get_samples()

    # Extract F, G, lambda_f, and lambda_g samples
    F_samples = posterior_samples["F"]
    G_samples = posterior_samples["G"]
    
    return F_samples, G_samples
    
    
def poisson_gamma_mcmc(training_data, latent_dim, warmup_steps, samples, mask_type = None):
    
    model = poisson_gamma_dynamical_system
    
    N,P = training_data.shape
    
    kernel = mcmc.NUTS(model)
    
    pyro.clear_param_store()
    mcmc_run = mcmc.MCMC(kernel, num_samples = samples, warmup_steps = warmup_steps)
    
    mcmc_run.run(training_data, latent_dim)
    
    posterior_samples = mcmc_run.get_samples()
    
    F_samples = posterior_samples["F"]
    
    
    G_samples = [posterior_samples[f"g_{t}"].numpy() for t in range(N)]
    G_samples = np.stack(G_samples, axis=1)
        
    return G_samples, F_samples

Now, we define our posterior check process, with both a posterior predictive check and a population predictive check. 

The population predictive check compares the likelihood of the current data samples of the produced probability distribution
to the likelihood of simulated samples from the produced probability distribution. 


The posterior predictive check is more robust, and rather masks out a portion of the data during the training process to be held out
for testing. For this masked part, we compare likelihood of the data samples that weren't used in training to likelihoods produced from 
simulating samples from the posterior probability distribution. 

We use 2 kinds of mask, a random mask, that creates a random combination of counties and timestamps to mask out, as well as
an end block mask, that masks out a lower rectangle, simulating holding out data for the last few time periods for a certain amount of 
counties, which better fits our task in time inference, as it is using past data to predict future data in a sense.

In [5]:
#General Population Predictive Check function

def population_predictive_check(reconstructed_mean, mask, data, model, variance=None): #Model Refers to PCA, Poisson, Etc
    log_prob = []
    test_prob_list = []

    for mean in reconstructed_mean:
        testing_mean = torch.tensor(mean[mask])
        
        if (model == 'Poisson'):
            sample_dist = dist.Poisson(testing_mean)
            
        elif (model == 'PCA'):
            sample_dist = dist.Normal(testing_mean, variance)
        
        else:
            print("Invalid model choice")
            return

        #Only have prediction on selected mask_rows and mask_cols
        y_pred  = sample_dist.sample()

        #Calculate the likelihood given this sample
        y_pred_likelihood = sample_dist.log_prob(y_pred).sum()

        #Now, we want to calculate the likelihood of the actual data. 
        data_test = torch.tensor(data[mask])

        test_prob = sample_dist.log_prob(data_test).sum()

        log_prob.append(-1 * y_pred_likelihood) 
        test_prob_list.append(-1 * test_prob)


    count = sum([1 for x, y in zip(log_prob, test_prob_list) if x > y]) #Count is if sample > data. 
    percent_likelihood = count / len(log_prob)

    print("Percentage of Test distribution more likely than Y_Pred is" + str(percent_likelihood))
    
    print(log_prob[:5])
    print(test_prob_list[:5])
    
    return percent_likelihood
    
    
#General Posterior Predictive Check Method(No Mask)

def posterior_predictive_check(reconstructed_mean, data, model, variance=None): #Model Refers to PCA, Poisson, Etc
    log_prob = []
    test_prob_list = []

    for mean in reconstructed_mean:
        if (model == 'Poisson'):
            sample_dist = dist.Poisson(mean)
            
        elif (model == 'PCA'):
            sample_dist = dist.Normal(mean, variance)
        
        else:
            print("Invalid model choice")
            return

        #Only have prediction on selected mask_rows and mask_cols
        y_pred = sample_dist.sample()

        #Calculate the likelihood given this sample
        y_pred_likelihood = sample_dist.log_prob(y_pred).sum()

        #Now, we want to calculate the likelihood of the actual data. 
        data_test = torch.tensor(data)

        test_prob = sample_dist.log_prob(data_test).sum()

        log_prob.append(-1 * y_pred_likelihood) 
        test_prob_list.append(-1 * test_prob)


    count = sum([1 for x, y in zip(log_prob, test_prob_list) if x > y]) #Count is if sample > data. 
    percent_likelihood = count / len(log_prob)

    print("Percentage of Test distribution more likely than Y_Pred is" + str(percent_likelihood))
    
    print(log_prob[:5])
    print(test_prob_list[:5])
    
    return percent_likelihood
    

Finally, when our model does pass the predictive check, we implement the actual synthetic control process. 

In this process, we take the trained samples that is indexed by counties (F_samples for Poisson, W_samples for PCA) ,
and perform a large scale regression using these samples. 

We regress the post intervention time stamp results on our X, which is a concatenation of the Trained samples and the 
actual data for the donor counties, and a A column that represents whether or not the county opened to fans that is 1 for the stadium county. 

To produce the synthetic control coutnerfactuals, we change the A from 1 to 0 for the stadium county, and then reconstruct 
the data with the new regression weights. 

In [64]:
from sklearn.linear_model import LinearRegression

def Synthetic_control(W_samples, donor_train, stadium_train, donor_test, stadium_test, dates, team, RBSC, regression_type = 'Ridge'):
    #W_Samples should be size latent_dim * p
    
    #Z_samples size n * latent_dim
    #Donor_data should be size n*p 
    
    #Stadium_data should be size n*1
    
    #Our Y should be donor_test and stadium_test, so size should be post_intervention_t * p+1
    #Stadium_test is our Y, shape p by T
    
    synthetic_control_paths = []
    pre_intervention_days, counties = donor_train.shape
    post_intervention_days, counties = donor_test.shape

    latent_dim, w_counties = W_samples[0].shape
    
    #Store results for different synthetic control regressions, as we iterate through W_samples
    counterfactual_outcomes_deconfound = []
    time_step_outcomes = []
    
    for W_sample in W_samples: 
    
        #Run Regression with latent_p W_sample on Y_it, for each timestep t post intervention. 
        
        
        #Add [1] if it is intervention
        
        A = torch.zeros(w_counties)
        A[-1] = 1
        A = A.view(1, w_counties)

        x_regressors = torch.cat((W_sample, A), dim=0).transpose(0,1)
        
        assert x_regressors.shape == (w_counties, latent_dim + 1)

        #For Yit ~ Ai + Yi(-m) + … + Yi(-1), ,we need stadium data pre intervention, which is just stadium train

        #We want to fit donor_test and stadium_test together in one regression
        stadium_test = stadium_test.view(-1, 1)

        Y = torch.cat((donor_test, stadium_test), dim = 1)
        
        
        assert Y.shape == (post_intervention_days,  w_counties)
        
        #Ridge regression with alpha = 0.001 Works better
        #Try Ridge, Lasso, MLP Regression ,with Cross Validation to find best value. 
        
        #Parameters for GridsearchCV for Lasso and Ridge
        parameters = {"alpha": [0, 1e-4,1e-3, 1e-2]}
        if regression_type == 'Ridge':
            reg = linear_model.Ridge()
            clf = GridSearchCV(reg, parameters, scoring='r2', cv=5)
        elif regression_type == 'MLP': 
            clf = MLPRegressor(hidden_layer_sizes=(20, 20), max_iter=5000)
        elif regression_type == 'Lasso':
            reg = linear_model.Lasso()
            parameters = {'alpha': [0.01]}
            #Right now 0.01 working best for lasso
            clf = GridSearchCV(reg, parameters, scoring = 'r2', cv=5)
            
        deconfound_lin_reg = clf.fit(x_regressors, Y.transpose(0,1))
        deconfound_lin_reg = clf.best_estimator_
        


        
        counterfactual_x = x_regressors
        counterfactual_x[w_counties-1, latent_dim] = 0 #Set "stadium open status" to 0.
        
        if regression_type == 'MLP':
            counterfactual_outcome = deconfound_lin_reg.predict(counterfactual_x)
        elif regression_type in ['Ridge', 'Lasso']:
            coefficients = torch.tensor(deconfound_lin_reg.coef_)
        
            assert coefficients.shape[1] == latent_dim + 1
            assert coefficients.shape[0] == post_intervention_days


            #Now, we perform reconstruction for the deconfounded counterfactual result by setting A=0 for the 
            #stadium county .

            counterfactual_outcome = counterfactual_x @ torch.tensor(coefficients).transpose(0,1)
        
        assert counterfactual_outcome.shape == Y.transpose(0,1).shape
        
        counterfactual_outcomes_deconfound.append(counterfactual_outcome)
        
        
        stadium_train = stadium_train.view(-1,1)

        
    
    if regression_type in ['Ridge', 'Lasso']:
        stadium_cf_deconfound = [counterfactuals[-1, :].numpy() for counterfactuals in counterfactual_outcomes_deconfound]
    else:
        stadium_cf_deconfound = [counterfactuals[-1, :] for counterfactuals in counterfactual_outcomes_deconfound]
    
    
    percentiles_5 = np.percentile(stadium_cf_deconfound, 5, axis=0)
    percentiles_95 = np.percentile(stadium_cf_deconfound, 95, axis=0)
    
    
    median_values = np.median(stadium_cf_deconfound, axis=0)
    post_dates = dates[-post_intervention_days:]
    plt.fill_between(post_dates, percentiles_5, percentiles_95, color='gray', alpha=0.3, label='5th-95th Percentile Range')
    pre_dates = dates[:pre_intervention_days]
    plt.plot(post_dates, median_values, label='Bayesian Synthetic Control Median Values ' + str(regression_type) + ' '+ str(team))
    plt.xticks(rotation=45)
    plt.tick_params(axis='x', which='major')
    
    plt.plot(post_dates, stadium_test, label = 'Stadium Data Postintervention')
    plt.plot(pre_dates, stadium_train, label = 'Stadium Data Pre-intervention')
    plt.axvline(x=dates[pre_intervention_days], color='grey')

    
    plt.plot(dates, RBSC, label = 'Robust Synthetic Control')
    plt.title('Synthetic Control ' + str(team) + " with number of latents " + str(W_samples[0].shape[0]) + " and counties " + str(W_samples[0].shape[1]) +" and number of samples " + str(len(W_samples)))
    plt.legend()
    plt.show()
    
    
    
    
    return counterfactual_outcomes_deconfound

# def Poisson_synthetic_control(F_samples, donor_train, stadium_train, donor_test, stadium_test, dates, team, RBSC, regression_type = 'Ridge'):
#     #F_samples should have shape latent_dim * donors

    
#     #Same thing as PCA synthetic control, except we use F_samples as deconfounding latents. 
#     counterfactuals = PCA_synthetic_control(F_samples, donor_train, stadium_train, donor_test, stadium_test, dates, team, RBSC, regression_type)
    
#     return counterfactuals

In [119]:
#Create dictionary for latent space dimensions for each team. 
latent_dim_dict = {'Arizona': 5, 'Atlanta':5 , 'Baltimore':15, 'Buffalo':25, 'Carolina': 5, 'Chicago':15, 'Cincinnati':13, 'Cleveland':1, 'Dallas': 37, 'Denver':5 , 'Detroit':12, 'Green Bay': 52, 'Houston': 12, 'Indianapolis': 10, 'Jacksonville':12, 'Kansas City':11, 'Las Vegas':12, 'LA Chargers':5,'LA Rams':8, 'Miami':8, 'Minnesota':5, 'New England': 9, 'New Orleans':18 , 'NY Giants':7,'NY Jets':7, 'Philadelphia':10, 'Pittsburgh': 25, 'San Francisco': 10, 'Seattle': 14, 'Tampa Bay': 15, 'Tennessee': 13, 'Washington':4}


In [ ]:
zipped_input = zip(grouped_df['Team'], grouped_df['County(s)'], grouped_df['State'], grouped_df['First date home stadium open to fans'])
import ast

def bayesian_SC_graph(team_county_data):
    count = 0
    for i, (team, county, state, date) in enumerate(team_county_data):
        count += 1
        date = ast.literal_eval(date)
        #team = ast.literal_eval(team)
        county = ast.literal_eval(county)
        state = ast.literal_eval(state)
        if team in ['San Francisco']:  #Ignore Arizona
            #x are donors, y are stadiums, train is pre-intervention, test is post intervention

        
            x_train, y_train, x_test, y_test, dates = get_training_data(team,county ,state, date, True, 0)

            x_train, y_train, x_test, y_test = torch.tensor(x_train.values),torch.tensor(y_train.values), torch.tensor(x_test.values), torch.tensor(y_test.values)


            #Combine Donor and Stadium in our traning data
            synthetic_control_training = torch.cat((x_train, torch.unsqueeze(y_train,1)), dim=1)

            #Produce Confounder variable to use in our regression

            #Run Poisson Factorization Model

            latent_dimension = latent_dim_dict[team]
            


            f_samples, g_samples = poisson_mcmc(synthetic_control_training, latent_dimension, 1500,1500, "None")

#             np.save(f'{team}_g_samples.npy', g_samples.numpy())
#             np.save(f'{team}_f_samples.npy', f_samples.numpy())
            synthetic_control_training = synthetic_control_training.clamp(min=0)
            
            
            N,P = synthetic_control_training.shape
            end_block_mask = torch.ones(N,P)
            end_block_mask[-30:, -3:] = 0 #The mask size can be edited
            end_block_mask = end_block_mask.bool()

            population_check = population_predictive_check(f_samples @ g_samples, end_block_mask, synthetic_control_training.round().to(torch.int64), 'Poisson')

        
            poisson_p_value = posterior_predictive_check(f_samples @ g_samples, synthetic_control_training.round().to(torch.int64),'Poisson')

            poisson_posterior_check = poisson_p_value >= 0
            
            RBSC_results = np.load(f'RBSC_Replications/{team}_RBSC.npy')
            
            if poisson_posterior_check:

                deconfounders_ridge = Synthetic_control(g_samples, x_train, y_train, x_test, y_test, dates, team, RBSC_results, regression_type='Ridge')
                #deconfounders_lasso = Synthetic_control(g_samples, x_train, y_train, x_test, y_test, dates, team, RBSC_results, regression_type='Lasso')

            else:
                w_samples, sigma_samples, z_samples = bayesian_pca_mcmc(synthetic_control_training, 10, 1000, 800)

                pca_p_value = posterior_predictive_check(z_samples@w_samples, synthetic_control_training, 'PCA', variance=sigma_samples[0])

                #With these samples, perform a POSTERIOR CHECK first, if it passes then we use that model. 

                pca_posterior_check = pca_p_value>=0

                if pca_posterior_check:
                    deconfounders = Synthetic_control(w_samples, x_train, y_train, x_test, y_test, dates, team, RBSC_results, regression_type='Ridge')



    return

bayesian_SC_graph(zipped_input)
        

Sample:  59%|█████▊    | 1757/3000 [1:42:45,  3.50s/it, step size=5.87e-04, acc. prob=0.930]

In [ ]:
zipped_input = zip(grouped_df['Team'], grouped_df['County(s)'], grouped_df['State'], grouped_df['First date home stadium open to fans'])

def bayesian_SC_graph_pre_loaded(team_county_data, calculate_ATT = False): #Difference is this function takes from pre-trained samples.
#We want to calculate ATT. 

    ATT_Bayesian_list = []
    ATT_RBSC_list = []
    
    for i, (team, county, state, date) in enumerate(team_county_data):
        #if team in ['Cincinnati', 'Cleveland', 'Carolina', 'Atlanta', 'Denver', 'Kansas City', 'Minnesota', 'Philadelphia', 'Seattle', 'San Francisco', 'Tennessee']:
        if team != 'Arizona': #Arizona data not included
            #f_samples = torch.from_numpy(np.load(f'{team}_f_samples.npy')) F samples used for posterior check
            g_samples = torch.from_numpy(np.load(f'Trained_Samples/{team}_g_samples.npy'))
            #f_samples = torch.from_numpy(np.load(f'Trained_Samples/{team}_f_samples.npy'))

            x_train, y_train, x_test, y_test, dates = get_training_data(team,county ,state, date, True, 0)

            x_train, y_train, x_test, y_test = torch.tensor(x_train.values),torch.tensor(y_train.values), torch.tensor(x_test.values), torch.tensor(y_test.values)
            synthetic_control_training = torch.cat((x_train, torch.unsqueeze(y_train,1)), dim=1)
            synthetic_control_training = synthetic_control_training.clamp(min=0)

            synthetic_control_training = synthetic_control_training.clamp(min=0)
            RBSC_results = create_synthetic_graph(team, county, state, date, False, -1)

            deconfounders_ridge = Poisson_synthetic_control(g_samples, x_train, y_train, x_test, y_test, dates, team, RBSC_results, regression_type='Ridge', save_fig = True)
            #deconfounders_lasso = Poisson_synthetic_control(g_samples, x_train, y_train, x_test, y_test, dates, team, RBSC_results, regression_type='Lasso')
            
            #post_check = posterior_predictive_check(f_samples @ g_samples, synthetic_control_training.round().to(torch.int64),'Poisson')
            
            if calculate_ATT:
                ATT_RBSC_list.append(RBSC_results)
                ATT_Bayesian_list.append(deconfounders_ridge)
                
#             posterior_check = posterior_predictive_check(f_samples @ g_samples, synthetic_control_training.round().to(torch.int64),'Poisson')
#             print(posterior_check)
    
#     min_len = min(len(lst) for lst in ATT_Bayesian_list)
#     truncated_lists = [lst[:min_len] for lst in ATT_Bayesian_list]
#     Bayesian_transposed= np.array(truncated_lists).T
#     ATT_Bayesian = np.mean(Bayesian_transposed, axis=0)
    
#     min_len = min(len(lst) for lst in ATT_bayesian_list)
#     truncated_lists = [lst[:min_len] for lst in ATT_Bayesian_list]
#     RBSC_transposed= np.array(truncated_lists).T
#     ATT_RBSC = np.mean(Bayesian_transposed, axis=0)
    
    return 1,2# ATT_Bayesian, ATT_RBSC
            
ATT_Bayesian, Att_RBSC = bayesian_SC_graph_pre_loaded(zipped_input, False)